<a href="https://colab.research.google.com/github/praveenpankaj/spatio-temporal-segmentation/blob/main/PASTIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ⚙️ Installing Required Dependencies

In [ ]:
# Move this to requirements.txt
print("⚙️ Installing dependencies...")
!pip install torchnet --quiet
!pip install torch_scatter --quiet

⚙️ Installing dependencies...
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.4 MB/s eta 0:00:00
  Created wh

## 📥 Downloading PASTIS Data

In [1]:
# Uncomment the following line to download the dataset if it's not already present
!wget https://zenodo.org/records/5012942/files/PASTIS.zip


--2025-02-23 15:31:28--  https://zenodo.org/records/5012942/files/PASTIS.zip
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.48.194, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28760245504 (27G) [application/octet-stream]
Saving to: ‘PASTIS.zip’

PASTIS.zip          100%[===================>]  26.78G  14.2MB/s    in 32m 24s 

2025-02-23 16:03:54 (14.1 MB/s) - ‘PASTIS.zip’ saved [28760245504/28760245504]



In [2]:
# Unzipping the dataset
print("📂 Extracting PASTIS dataset...")
!unzip -q PASTIS.zip


📂 Extracting PASTIS dataset...


In [6]:
# uncomment this line to remove the downloaded archive file to save space
!rm -r '/content/PASTIS.zip'

## 🔗 Mounting Google Drive in Colab

In [3]:
import os
import json
import numpy as np
import shutil
from google.colab import drive


if not os.path.exists('outpath'):
  os.mkdir('outpath')

In [5]:
# Mount Google Drive to access and store data
print("🔗 Mounting Google Drive...")
drive.mount('/content/drive')

🔗 Mounting Google Drive...
Mounted at /content/drive


## 🚚 Transferring PASTIS data from Google Colab to Google Drive

In [ ]:
# uncomment for the files to be moved permanently into the google drive from colab
## Define source and destination paths
#src_path = '/content/outpath'
#dst_path = '/content/drive/MyDrive/PASTIS_Data/'

## Move dataset from Colab storage to Google Drive
#print(f"🚛 Moving dataset to Google Drive: {dst_path}")
#shutil.move(src_path, dst_path)
#print("✅ File moved successfully.")

File moved to: /content/drive/MyDrive/PASTIS_Data/


## 📊 Sampling 100 Data Points from Each Region

Also need to change Tile names in metadata.geojson file to sample

In [10]:
# Path to the metadata file

geojson_file = '/content/PASTIS/metadata.geojson'

# List to store selected patch IDs
file_list = []

try:
    with open(geojson_file, 'r') as file:
        geojson_data = json.load(file)

    count = 0
    target_tile = "roi_pallab_200"  # Change as needed: "roi_pallab_100", "roi_pallab_200", "roi_pallab_300"

    for feature in geojson_data['features']:
        if 'TILE' in feature['properties']:
            if feature['properties']['TILE'] == target_tile:
                count += 1
                if count <= 100:
                    file_list.append(feature['properties']['ID_PATCH'])

    print(f"✅ Sampled {len(file_list)} patches from {target_tile}")

except Exception as e:
    print(f"❌ Error processing GeoJSON file: {e}")


✅ Sampled 0 patches from roi_pallab_200



## 🎛️ Noise Implementation - Adding Noise to 25% of Training Data

In [11]:
# Function to add Gaussian noise to image data
def add_gaussian_noise(data, mean=0, std=25):
    """Applies Gaussian noise to an image."""
    gauss = np.random.normal(mean, std, data.shape).astype('float32')
    return data + gauss

# Define directory containing the dataset
input_dir = '/content/PASTIS/DATA_S2'


# Define start and end indices for applying noise
start_idx = 10101
end_idx = 10200

print("🖼️ Applying Gaussian noise to selected images...")


# Iterate through dataset and apply noise to selected files
for filename in os.listdir(input_dir):
    if filename.endswith('.npy'):
        idx = int(filename.split('_')[1].split('.')[0])
        input_path = os.path.join(input_dir, filename)

        if os.path.isfile(input_path) and start_idx <= idx <= end_idx:
            data = np.load(input_path)
            noisy_data = add_gaussian_noise(data)
            np.save(input_path, noisy_data)
            print(f"🎨 Processed with noise: {input_path}")
        else:
            print(f"⚠️ Skipping: {input_path} (Outside range or missing)")

🖼️ Applying Gaussian noise to selected images...
⚠️ Skipping: /content/PASTIS/DATA_S2/S2_10465.npy (Outside range or missing)
⚠️ Skipping: /content/PASTIS/DATA_S2/S2_10425.npy (Outside range or missing)
⚠️ Skipping: /content/PASTIS/DATA_S2/S2_30090.npy (Outside range or missing)
⚠️ Skipping: /content/PASTIS/DATA_S2/S2_30714.npy (Outside range or missing)
⚠️ Skipping: /content/PASTIS/DATA_S2/S2_40324.npy (Outside range or missing)
⚠️ Skipping: /content/PASTIS/DATA_S2/S2_40552.npy (Outside range or missing)
⚠️ Skipping: /content/PASTIS/DATA_S2/S2_20398.npy (Outside range or missing)
⚠️ Skipping: /content/PASTIS/DATA_S2/S2_30527.npy (Outside range or missing)
⚠️ Skipping: /content/PASTIS/DATA_S2/S2_10456.npy (Outside range or missing)
⚠️ Skipping: /content/PASTIS/DATA_S2/S2_20272.npy (Outside range or missing)
⚠️ Skipping: /content/PASTIS/DATA_S2/S2_20026.npy (Outside range or missing)
⚠️ Skipping: /content/PASTIS/DATA_S2/S2_40288.npy (Outside range or missing)
⚠️ Skipping: /content/PASTI

## 🛠️ Cloning Required GitHub Repositories

In [12]:
print("📥 Cloning required repositories...")
!git clone https://github.com/VSainteuf/utae-paps.git

📥 Cloning required repositories...
Cloning into 'utae-paps'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 97 (delta 35), reused 24 (delta 24), pack-reused 47 (from 1)
Receiving objects: 100% (97/97), 3.11 MiB | 37.08 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [13]:
!git clone https://github.com/VSainteuf/pastis-benchmark.git

Cloning into 'pastis-benchmark'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 185 (delta 18), reused 17 (delta 17), pack-reused 160 (from 1)
Receiving objects: 100% (185/185), 9.24 MiB | 9.84 MiB/s, done.
Resolving deltas: 100% (91/91), done.


## 📂 Verifying Dataset Files in Drive

In [14]:
# Define data directory
data_dir = '/content/PASTIS'

# List all files in the directory
files = os.listdir(data_dir)

# Print the file names
print("📂 Files in dataset directory:", files)
#['NORM_S2_patch.json', 'ANNOTATIONS', 'DATA_S2', 'INSTANCE_ANNOTATIONS', 'metadata.geojson', '.ipynb_checkpoints']

📂 Files in dataset directory: ['NORM_S2_patch.json', 'ANNOTATIONS', 'INSTANCE_ANNOTATIONS', 'DATA_S2', 'metadata.geojson']


## 🏋️ Training and Inference

In [16]:
print("🚀 Training segmentation model...")
os.chdir('/content/utae-paps')
!python train_semantic.py --fold 1 --dataset_folder $data_dir --res_dir '/content/outpath'

print("✅ Training complete! Results saved to '/content/outpath'")

🚀 Training segmentation model...
Traceback (most recent call last):
  File "/content/utae-paps/train_semantic.py", line 17, in <module>
    import torchnet as tnt
ModuleNotFoundError: No module named 'torchnet'
✅ Training complete! Results saved to '/content/outpath'
